### Naive and Aware Times

We decided early on that we would only work with naive times in UTC.

But we still need the ability to transform aware times into naive UTC times, and vice versa.

If no other reason that we will be ingesting datetime data that may contain timezone information.

We'll come back to a 3rd party library that can help us substantially with this, but for now, let's use plain Python to understand its basic timezone functionality.

When we parsed a `datetime` object from an ISO string, we saw that we could end up with a time zone aware `datetime`:

In [1]:
s = "2020-03-15T13:30:00-07:00"

In [2]:
from datetime import datetime

In [3]:
datetime.fromisoformat(s)

datetime.datetime(2020, 3, 15, 13, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))

As you can see, that `datetime` object has a `tzinfo` property that is not `None` - it is a `timezone` object, so it is timezone aware.

In [4]:
dt = datetime.fromisoformat(s)

In [5]:
dt.tzinfo

datetime.timezone(datetime.timedelta(days=-1, seconds=61200))

As we can see, the time zone was expressed as a UTC offset - an offset is nothing more than a duration, and the timezone offset definition is actually expressed using a `timedelta` object.

Let's look at `timezone` objects.

A Python `timezone` object is nothing more than a `name` associated with a `timedelta` object that represents the UTC offset.

The idea is that if we have an aware datetime with a timezone, we simply add the timezone's `timedelta` object to a naive version of the datetime, and this gives us a naive UTC timestamp.

For example, if I am located in New York, and we are in DST, our time zone is EDT (as opposed to EST if we are not in DST).

So, I could create a timezone named `EDT`, with an offset from UTC of -4 hours:

In [6]:
from datetime import timezone, timedelta

In [7]:
tz_EDT = timezone(timedelta(hours=-4), 'EDT')

In [8]:
tz_EDT

datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')

The `timezone` class also has a UTC timezone pre-defined:

In [9]:
timezone.utc

datetime.timezone.utc

Let's define another time zone, Central Daylight Time (CDT) this time - it has a -5 hour offset from UTC:

In [10]:
tz_CDT = timezone(timedelta(hours=-5), 'CDT')

Now let's start with an aware datetime in EDT:

In [11]:
dt = datetime(year=2020, month=5, day=15, hour=22, minute=30, tzinfo=tz_EDT)

In [12]:
dt

datetime.datetime(2020, 5, 15, 22, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT'))

We want to "convert" this datetime into the equivalent datetime but in CDT time zone.

We can use the `.astimezone()` method that `datetime` objects have:

In [13]:
dt.astimezone(tz_CDT)

datetime.datetime(2020, 5, 15, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

Notice how we end up with an aware datetime, but with a CDT time zone - and notice how the new time reflects the time zone change (one hour earlier than EDT).

Similarly, we can convert an aware datetime of any time zone into an aware datetime in UTC:

In [14]:
dt_utc = dt.astimezone(timezone.utc)

In [15]:
dt_utc

datetime.datetime(2020, 5, 16, 2, 30, tzinfo=datetime.timezone.utc)

Again notice how the time zone now shows UTC, and the date/time was adjusted accordingly.

So now we are able to convert an aware datetime from one time zone to another.

But now that we have that datetime in UTC, how do we make it naive?

Remember that a datetime is naive if it has no time zone information, i.e. `tzinfo` is `None`.

We can use the `replace` method on our `dt_utc` object, that creates a new `datetime` object, copying all the values over, except the ones we specify as a replacement.

In this case, we want to copy everything into a new `datetime` except the timezone, which we want to make `None`:

In [16]:
dt_naive = dt_utc.replace(tzinfo=None)

In [17]:
dt_naive

datetime.datetime(2020, 5, 16, 2, 30)

So now we've seen how to take an aware datetime with some timezone, and convert it to a naive datetime in UTC - let's recap:

In [18]:
s = "2020-05-15T13:30:00-04:00"

dt_aware = datetime.fromisoformat(s)
dt_utc = dt_aware.astimezone(timezone.utc)
dt_naive = dt_utc.replace(tzinfo=None)

print(dt_naive.isoformat())

2020-05-15T17:30:00


Now we want to do the opposite - convert a naive UTC datetime into an aware datetime in some other time zone. The process is basically the same:

In [19]:
dt_naive = datetime.fromisoformat('2020-05-15T17:30:00')

In [20]:
dt_aware = dt_naive.replace(tzinfo=timezone.utc)

In [21]:
dt_aware

datetime.datetime(2020, 5, 15, 17, 30, tzinfo=datetime.timezone.utc)

In [22]:
tz_EDT = timezone(timedelta(hours=-4), 'EDT')
tz_CDT = timezone(timedelta(hours=-5), 'CDT')

In [23]:
dt_aware.astimezone(tz_EDT)

datetime.datetime(2020, 5, 15, 13, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT'))

In [24]:
dt_aware.astimezone(tz_CDT)

datetime.datetime(2020, 5, 15, 12, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

One thing in all this, is that we never had to deal with DST - we simply defined our timezone as a UTC offset, and applied the particular time zone - here I used EDT and CDT (instead of EST and CST) because I **knew** the particular date we were looking at was during DST. In general, you don't know that though - and this adds another level of complexity - when converting a UTC datetime to Eastern, should you pick EST or EDT? What about all the other timezones in the world? Not everyone changes over DST on the same day - and some don't even change, ever (like Phoenix Arizona for example). Not only that, but DST changes are not always constant - they can, and have, varied historically, for the same time zone - so a history needs to be maintained, and of course things will change in the future.

Dealing with timezones is bad enough, but DST is 10x worse!

Anyone interested in starting a worldwide petition to drop daylight savings times??!! :-)

For that reason, there is a standarized database that captures all this information - time zone names, UTC offsets, DST rules, etc, called the Olson Database (named after the original creator) - also known as the IANA timezone database: https://en.wikipedia.org/wiki/Tz_database

We'll look at some 3rd party libraries later in this course that will simplify our life a lot!